# CENSUS DATA WRANGLING

## Guide

1. **Step 1: Load Packages, Functions, and Data Dictionary**
2. **Step 2: Find Variables Using Data Dictionary**
3. **Step 3: Fetch Raw Block Group Data From Census API**
4. **Step 4: Engineer and Format Features Using Raw Data**
- **Appendix A - EXTRA FEATURE: Variable Key Search Using `censusdata` Package** <br>
- **Appendix B - EXTRA FEATURE: Extracting Zip Code Data**

### Step 1: Load Packages, Functions, and Data Dictionary

In [226]:
def standardize_col_names(df):
    '''
    Standardizes column names of a dataframe.
    It will remove white space, replace spaces with underscores, and eliminate special characters (including parenthesis and slashes).
    
    Parameters
    ----------
    
    df : Dataframe object
    
    Return Values
    -------------
    Dataframe with column names standardized.
    '''
    df.columns = (df.columns
                .str.strip()
                .str.lower()
                .str.replace(' ', '_')
                .str.replace(':', '_')
                .str.replace('(', '')
                .str.replace(')', '')
                .str.replace('/','')
                .str.replace('\\',''))
    return df


In [237]:
def numerizer(three_digit_string):
    """
    transforms string of three digits into a number
    """
    return three_digit_string[0]*100 + three_digit_string[1]*10 + three_digit_string[2]

In [239]:
def range_creator(head, beginning, end):
    """
    creates full list of variables to retrieve from var_xxx-var_xxx
    """
    more_than_99 = [head + str(i) + 'E' for i in range(beginning, end+1) if len(str(i))==3]
    less_than_100s = [head + '0' + str(i) + 'E' for i in range(beginning, end+1) if len(str(i))==2]
    less_than_10s = [head + '00' + str(i) + 'E' for i in range(beginning, end+1) if len(str(i))==1]
    full_range = less_than_10s + less_than_100s + more_than_99
    return full_range

In [279]:
def sum_extract_divide(var, splitter='-'):
    """
    extracts list of variables to add, adds them, and divides the sum by the denominator var[1]
    """
    digits_0 = [int(s) for s in var[0].split('+')[0].split(splitter)[0][-3:] if s.isdigit()]
    digits_1 = [int(s) for s in var[0].split('+')[0].split(splitter)[1][-4:] if s.isdigit()]
    digits_2 = [int(s) for s in var[0].split('+')[1].split(splitter)[0][-3:] if s.isdigit()]
    digits_3 = [int(s) for s in var[0].split('+')[1].split(splitter)[1][-5:] if s.isdigit()]
    beginning_a = numerizer(digits_0)
    end_a = numerizer(digits_1)
    beginning_b = numerizer(digits_2)
    end_b = numerizer(digits_3)
    first_range = range_creator(var[0].split('+')[0][-11:-4], beginning_a, end_a)
    second_range = range_creator(var[0].split('+')[1][-12:-5], beginning_b, end_b)
    first_total = df[first_range].sum(axis=1)
    second_total = df[second_range].sum(axis=1)
    try:
        output = (first_total + second_total)/df[var[1]+'E']
    except ZeroDivisionError as err:
        output = 0

In [2]:
# install package for wrangling census data
!pip install censusdata

In [311]:
import re
import numpy as np
import pandas as pd
import censusdata
import os
import sys
from tqdm.notebook import tqdm # progress bar for step 3

In [261]:
# import api key to avoid request limits (needed if getting all block group data); get from here https://www.census.gov/developers/ (takes 1-5 min)
with open('api-key.txt', 'r') as file:
    api_key = file.read()

In [316]:
src_path = os.path.abspath('../src/data/')
sys.path.append(src_path)

from acs_getter import *

In [313]:
 # setup input and output locations
input_loc = '../data/processed/'
output_loc = '../data/processed/'

### Step 2: Find Variables Using Data Dictionary

In [314]:
# read in data dictionary excel sheet
data_dictionary = pd.read_excel (input_loc + 'ACS Block Group Data Dictionary.xlsx', header=8, usecols="A:G")

In [321]:
# standardize column names
data_dictionary = standardize_col_names(data_dictionary)
data_dictionary

,metric,acs_variables_used,variable_name_in_dataset,calculated_,variable_main_level_description,variable_sub-level_description,variable_population_as_described_by_the_census
0,Income,B19301_001,inc_pcincome,B19301_001,Per Capita Income in the Past 12 Months (In 20...,Per Capita Income in the Past 12 Months (In 20...,Total Population
1,Income,B17021_001,tot_population,B17021_001,Poverty Status of Individuals in the Past 12 m...,Total,Population for which poverty status can be det...
2,Income,B17021_002,inc_pct_poverty,B17021_002/B17021_001,Poverty Status of Individuals in the Past 12 m...,Income in the past 12 months is below poverty ...,Population for which poverty status can be det...
3,Race,B02001_001,NaN,B02001_001,Race,Total,Total Population
4,Race,B02001_002,race_pct_white,B02001_002/B02001_001,Race,White Alone,Total Population
...,...,...,...,...,...,...,...
121,SNAP and Disability,B22010_001,total_households,B22010_001,Receipt of Food Stamps by Disability,Total,Households
122,SNAP and Disability,B22010_002,pct_snap_past_12mo,B22010_002/B22010_001,Receipt of Food Stamps by Disability,Households that Received Food Stamps/SNAP in t...,Households
123,SNAP and Disability,"B22010_003, B22010_006",pct_disability,(B22010_003 + B22010_006) / B22010_001,Receipt of Food Stamps by Disability,Households with 1 or more persons with a disab...,Households
124,Median Number of Rooms,B25018_001,median_room_num,B25018_001,Median Number of Rooms,Median Number of Rooms,Housing Units


In [262]:
# extract variables from data dictionary (NOTE uses constants which must be adjusted if new variables are added to dictionary; will fix)
df2 = data_dictionary.acs_variables_used.str.split(', ').apply(pd.Series)
df2.index = data_dictionary.set_index(['metric', 'variable_name_in_dataset']).index
acs_variables = df2.stack().reset_index(['metric', 'variable_name_in_dataset'])[0]
acs_variables = acs_variables.reset_index()
acs_variables = acs_variables.drop(columns='index')
acs_variables = acs_variables[0]
variables_list = list(acs_variables[:11])
for var in acs_variables[11:17]:
    for i in range(int(var[8:10]), int(var[-2:])+1):
        if i < 10:
            new_var = var[:7]+'00'+str(i)
            variables_list.append(new_var)
        else:
            new_var = var[:7]+'0'+str(i)
            variables_list.append(new_var)
variables_list += list(acs_variables[17:])
variables_list = [x + 'E' for x in variables_list]
variables_list

In [266]:
variables_list = [x + 'E' for x in variables_list]

### Step 3: Fetch Raw Block Group Data From Census API

We can't use the '\*' for states or counties if we're trying to get block group level data so we have to create some lists. We also have to use an API key due to the volume of requests.

In [269]:
# create state codes
state_codes = {
    'WA': '53', 'DE': '10', 'DC': '11', 'WI': '55', 'WV': '54', 'HI': '15',
    'FL': '12', 'WY': '56', 'NJ': '34', 'NM': '35', 
    'LA': '22', 'NC': '37', 'ND': '38', 'NE': '31', 'TN': '47', 'NY': '36',
    'PA': '42', 'AK': '02', 'NV': '32', 'NH': '33', 'VA': '51',  
    'AL': '01', 'AR': '05', 'VT': '50', 'IL': '17', 'GA': '13',
    'IN': '18', 'IA': '19', 'MA': '25', 'AZ': '04', 'ID': '16', 'CT': '09',
    'ME': '23', 'MD': '24', 'OK': '40', 'OH': '39', 'UT': '49', 'MO': '29',
    'MN': '27', 'MI': '26', 'RI': '44', 'KS': '20', 'MT': '30', 'MS': '28',
    'SC': '45', 'KY': '21', 'OR': '41', 'SD': '46', 'CO': '08',
    'CA': '06',
    'TX': '48',
}

# Create list of numerical codes
all_states = list(state_codes.values())

In [270]:
%%time
# extract raw data for all states from Census API
# 2 NOTES: 
# - 1) This requires an API key 
# - 2) This can take several hours

year = 2018 # <-- this is the latest year available, but can be set to an earlier year

counter = 1
df = pd.DataFrame()
for state in tqdm(all_states):
    print(f"task {counter}: compiling block group data from state #{state}")
    counties = [county[1].params()[1][1] for county in list(censusdata.geographies(censusdata.censusgeo([('state', state), ('county', '*')]), 'acs5', 2018, key=api_key).items())]
    for county in tqdm(counties):
        county_df = censusdata.download('acs5', year,
                             censusdata.censusgeo([('state', state), ('county', county), ('block group', '*')]),
                             variables_list, key=api_key)
        df = df.append(county_df)
    counter += 1
df.head()

task 1: compiling block group data from state #53



task 2: compiling block group data from state #10



task 3: compiling block group data from state #11



task 4: compiling block group data from state #55



task 5: compiling block group data from state #54


...


task 49: compiling block group data from state #08



task 50: compiling block group data from state #06



task 51: compiling block group data from state #48




Wall time: 4h 12min 52s


,B19301_001E,B17021_001E,B17021_002E,B02001_001E,B02001_002E,B02001_003E,B02001_004E,B03002_001E,B03002_003E,B03002_012E,...,B25081_004E,B25081_005E,B25081_006E,B25081_008E,B22010_001E,B22010_002E,B22010_003E,B22010_006E,B25018_001E,B25035_001E
"Block Group 1, Census Tract 901.02, Kitsap County, Washington: Summary level: 150, state:53> county:035> tract:090102> block group:1",38870.0,2655,226,2677,2280,0,11,2677,2247,101,...,17,76,0,191,1144,126,78,142,6.0,1991
"Block Group 1, Census Tract 919, Kitsap County, Washington: Summary level: 150, state:53> county:035> tract:091900> block group:1",26235.0,2419,265,2440,1481,168,28,2440,1328,205,...,0,8,8,185,896,98,86,273,6.0,1996
"Block Group 2, Census Tract 919, Kitsap County, Washington: Summary level: 150, state:53> county:035> tract:091900> block group:2",38522.0,705,49,705,542,0,0,705,542,16,...,5,0,0,137,341,10,0,170,5.8,1963
"Block Group 2, Census Tract 811, Kitsap County, Washington: Summary level: 150, state:53> county:035> tract:081100> block group:2",24710.0,1541,408,1635,1227,134,23,1635,1227,91,...,7,32,0,17,628,210,76,135,5.0,1945
"Block Group 2, Census Tract 802, Kitsap County, Washington: Summary level: 150, state:53> county:035> tract:080200> block group:2",18788.0,2029,572,2029,1371,162,14,2029,1341,187,...,0,8,0,74,866,366,216,208,4.4,1967


In [271]:
# save rawoutput_loca
df.to_csv(output_loc+ 'ACS_data_2018_raw.csv')

### Step 4: Engineer and Format Features Using Raw Data

In [319]:
# load raw data
df = pd.read_csv(input_loc + 'data_2018_raw.csv')

In [322]:
# extract non-empty data dictionary variable names and formulas
data_formulas = data_dictionary.dropna(subset=['acs_variables_used','variable_name_in_dataset'])

In [323]:
# feature engineer raw data 
print(f"--> this number: {df.shape[1] + data_formulas.shape[0]}")
for name, var in zip(data_formulas.variable_name_in_dataset,
                     data_formulas.calculated_.apply(lambda x: x.replace(" ", "").split('/'))): # splits formulas up by '/' sign to extract numerator (var[0]) and denominator(var[1])
    if len(var) == 1: # i.e. no engineering needed, just renaming
        print(name, var, "found only one variable")
        df[name] = df[var[0] + 'E']
    else:
        if var[0].find('+') != -1 and var[0].find('sum') == -1: # found + but no sum in the numerator
            print(name, var, "found + but no sum")
            numerator = var[0].replace("(","").replace(")", "").split('+')
            try:
                df[name] = (df[numerator[0] + 'E'] + df[numerator[1] + 'E'])/df[var[1] + 'E']
            except ZeroDivisionError as err:
                df[name] = 0
        elif var[0].find('+') != -1 and var[0].find('sum') != -1 and var[0].find(':') == -1: # found +, sum, but no : in the numerator
            print(name, var, "found +, sum, but no :")
            df[name] = sum_extract_divide(var)
        elif var[0].find('+') != -1 and var[0].find('sum') != -1 and var[0].find(':') != -1: # found +, sum, and : in the numerator
            print(name, var, "found +, sum, and :")
            df[name] = sum_extract_divide(var, splitter=':')
        elif var[0].find('+') == -1 and var[0].find('sum') != -1 and var[0].find(':') == -1: # found sum but no + in the numerator
            print(name, var, "found sum but no +")
            add_list = [x +'E' for x in var[0].replace('sum(', '').replace(')', '').split(',')]
            total = df[add_list].sum(axis=1)
            try:
                df[name] = total/df[var[1]+'E']
            except ZeroDivisionError as err:
                df[name] = 0
        else: # found only / in the numerator
            print(name, var, "found only /")
            try:
                df[name] = df[var[0] + 'E']/df[var[1] + 'E']
            except ZeroDivisionError as err:
                df[name] = 0
print(f"--> should match this number: {df.shape[1]}")

--> this number: 310
inc_pcincome ['B19301_001'] found only one variable
tot_population  ['B17021_001'] found only one variable
inc_pct_poverty ['B17021_002', 'B17021_001'] found only /
race_pct_white  ['B02001_002', 'B02001_001'] found only /
race_pct_black ['B02001_003', 'B02001_001'] found only /
race_pct_amid ['B02001_004', 'B02001_001'] found only /
race_pct_whitenh ['B03002_003', 'B03002_001'] found only /
race_pct_hisp ['B03002_012', 'B03002_001'] found only /
age_pct_under25 ['(sum(B01001_003-B01001_010)+sum(B01001_027-B01001_034))', 'B01001_001'] found +, sum, but no :
age_pct_25_64 ['(sum(B01001_011-B01001_019)+sum(B01001_035-B01001_043))', 'B01001_001'] found +, sum, but no :
age_pct_over65 ['(sum(B01001_020:B01001_025)+sum(B01001_044:B01001_049))', 'B01001_001'] found +, sum, and :
sex_tot_pop ['B01001_001'] found only one variable
pct_male  ['B01001_002', 'B01001_001'] found only /
pct_female ['B01001_026', 'B01001_001'] found only /
educ_tot_pop ['B15002_001'] found only 

In [324]:
# drop unused raw columns
formatted_df = pd.concat([df.iloc[:, 0],df.iloc[:, -data_formulas.shape[0]:]], axis=1)

In [325]:
# standardize column names
formatted_df = standardize_col_names(formatted_df)

In [326]:
# create geoid column
formatted_df.unnamed__0 = formatted_df.unnamed__0.apply(lambda x: x.replace(',', '').replace(':', '').replace('.', '').replace('>', ''))
formatted_df['geoid'] = formatted_df.unnamed__0.apply(lambda x:'#_' + ''.join(re.findall(r'\d+', x)[-4:])) #transform description into #_geoid

In [327]:
# reorder columns
cols = list(formatted_df.columns)
cols = [cols[-1]] + cols[:-1]
formatted_df = formatted_df[cols]

In [328]:
# confirm formatting
formatted_df

,geoid,unnamed__0,inc_pcincome,tot_population,inc_pct_poverty,race_pct_white,race_pct_black,race_pct_amid,race_pct_whitenh,race_pct_hisp,...,house_w_1_mort,house_w_2_mort,house_w_home_equity_loan,house_w_both_2_mort_and_loan,house_no_mort,total_households,pct_snap_past_12mo,pct_disability,median_room_num,median_year_built
0,#_530350901021,Block Group 1 Census Tract 90102 Kitsap County...,38870.0,2655.0,0.085122,0.851700,0.000000,0.004109,0.839372,0.037729,...,0.610425,0.023320,0.104252,0.000000,0.262003,1144,0.110140,0.192308,6.0,1991
1,#_530350919001,Block Group 1 Census Tract 919 Kitsap County W...,26235.0,2419.0,0.109549,0.606967,0.068852,0.011475,0.544262,0.084016,...,0.622180,0.000000,0.015038,0.015038,0.347744,896,0.109375,0.400670,6.0,1996
2,#_530350919002,Block Group 2 Census Tract 919 Kitsap County W...,38522.0,705.0,0.069504,0.768794,0.000000,0.000000,0.768794,0.022695,...,0.534426,0.016393,0.000000,0.000000,0.449180,341,0.029326,0.498534,5.8,1963
3,#_530350811002,Block Group 2 Census Tract 811 Kitsap County W...,24710.0,1541.0,0.264763,0.750459,0.081957,0.014067,0.750459,0.055657,...,0.791822,0.026022,0.118959,0.000000,0.063197,628,0.334395,0.335987,5.0,1945
4,#_530350802002,Block Group 2 Census Tract 802 Kitsap County W...,18788.0,2029.0,0.281912,0.675702,0.079842,0.006900,0.660917,0.092164,...,0.670683,0.000000,0.032129,0.000000,0.297189,866,0.422633,0.489607,4.4,1967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217734,#_481450007001,Block Group 1 Census Tract 7 Falls County Texa...,26364.0,1421.0,0.139338,0.901478,0.098522,0.000000,0.843068,0.058410,...,0.338863,0.000000,0.000000,0.000000,0.661137,476,0.058824,0.445378,5.4,1979
217735,#_481450007002,Block Group 2 Census Tract 7 Falls County Texa...,19821.0,1348.0,0.300445,0.869472,0.114836,0.000000,0.532097,0.352354,...,0.266667,0.027451,0.031373,0.000000,0.674510,377,0.238727,0.408488,5.1,1965
217736,#_481450008001,Block Group 1 Census Tract 8 Falls County Texa...,21012.0,830.0,0.100000,0.843862,0.033373,0.000000,0.692491,0.274136,...,0.194805,0.000000,0.000000,0.000000,0.805195,248,0.024194,0.322581,5.2,1962
217737,#_481450008002,Block Group 2 Census Tract 8 Falls County Texa...,11636.0,1547.0,0.311571,0.558356,0.417604,0.007367,0.412951,0.156262,...,0.430636,0.000000,0.000000,0.000000,0.569364,479,0.240084,0.471816,5.5,1975


In [330]:
# save formatted dataframe to csv
formatted_df.to_csv(output_loc + "ACS_data_2018_formatted.csv")

#### Raw Data Removal
CAUTION: It will require rerunning a multi-hour fetch command ("Step 3") to restore the raw data. Only do this if you are certain.

In [331]:
# CAUTION!!!! THIS REMOVES raw data csv to save space
os.remove(output_loc + "ACS_data_2018_raw.csv")
print("ACS_data_2018_raw.csv was removed!")

data_2018_raw.csv was removed!


#### Complete

A formatted csv of data dictionary variables for all block groups in the USA should be saved in the output folder

### Appendix A - EXTRA FEATURE: Variable Key Search Using `censusdata` Package:

In [301]:
# keywords to search for target variables
key_words = ['rural', 'urban', 'total population', 'density', 'poverty', 'income', 'labor', 'employment']

In [302]:
# search for all variables in by concept
for word in key_words:
    print(word)
    display(censusdata.search('acs5', 2018, 'concept', word)[0:10]) #restricted range as this is still buggy (seems to return extra results for 'GEOID' variable)

rural


[]

urban


[]

total population


...

[('B05010_001E',
  'RATIO OF INCOME TO POVERTY LEVEL IN THE PAST 12 MONTHS BY NATIVITY OF CHILDREN UNDER 18 YEARS IN FAMILIES AND SUBFAMILIES BY LIVING ARRANGEMENTS AND NATIVITY OF PARENTS',
  'Estimate!!Total'),
 ('B05010_002E',
  'RATIO OF INCOME TO POVERTY LEVEL IN THE PAST 12 MONTHS BY NATIVITY OF CHILDREN UNDER 18 YEARS IN FAMILIES AND SUBFAMILIES BY LIVING ARRANGEMENTS AND NATIVITY OF PARENTS',
  'Estimate!!Total!!Under 1.00'),
 ('B05010_003E',
  'RATIO OF INCOME TO POVERTY LEVEL IN THE PAST 12 MONTHS BY NATIVITY OF CHILDREN UNDER 18 YEARS IN FAMILIES AND SUBFAMILIES BY LIVING ARRANGEMENTS AND NATIVITY OF PARENTS',
  'Estimate!!Total!!Under 1.00!!Living with two parents'),
 ('B05010_004E',
  'RATIO OF INCOME TO POVERTY LEVEL IN THE PAST 12 MONTHS BY NATIVITY OF CHILDREN UNDER 18 YEARS IN FAMILIES AND SUBFAMILIES BY LIVING ARRANGEMENTS AND NATIVITY OF PARENTS',
  'Estimate!!Total!!Under 1.00!!Living with two parents!!Both parents native'),
 ('B05010_005E',
  'RATIO OF INCOME TO PO

labor


[('B12006_001E',
  'MARITAL STATUS BY SEX BY LABOR FORCE PARTICIPATION',
  'Estimate!!Total'),
 ('B12006_002E',
  'MARITAL STATUS BY SEX BY LABOR FORCE PARTICIPATION',
  'Estimate!!Total!!Never married'),
 ('B12006_003E',
  'MARITAL STATUS BY SEX BY LABOR FORCE PARTICIPATION',
  'Estimate!!Total!!Never married!!Male'),
 ('B12006_004E',
  'MARITAL STATUS BY SEX BY LABOR FORCE PARTICIPATION',
  'Estimate!!Total!!Never married!!Male!!In labor force'),
 ('B12006_005E',
  'MARITAL STATUS BY SEX BY LABOR FORCE PARTICIPATION',
  'Estimate!!Total!!Never married!!Male!!In labor force!!Employed or in Armed Forces'),
 ('B12006_006E',
  'MARITAL STATUS BY SEX BY LABOR FORCE PARTICIPATION',
  'Estimate!!Total!!Never married!!Male!!In labor force!!Unemployed'),
 ('B12006_007E',
  'MARITAL STATUS BY SEX BY LABOR FORCE PARTICIPATION',
  'Estimate!!Total!!Never married!!Male!!Not in labor force'),
 ('B12006_008E',
  'MARITAL STATUS BY SEX BY LABOR FORCE PARTICIPATION',
  'Estimate!!Total!!Never married

employment


[('B10058_001E',
  'EMPLOYMENT STATUS OF GRANDPARENTS LIVING WITH OWN GRANDCHILDREN UNDER 18 YEARS BY RESPONSIBILITY FOR OWN GRANDCHILDREN AND AGE OF GRANDPARENT',
  'Estimate!!Total'),
 ('B10058_002E',
  'EMPLOYMENT STATUS OF GRANDPARENTS LIVING WITH OWN GRANDCHILDREN UNDER 18 YEARS BY RESPONSIBILITY FOR OWN GRANDCHILDREN AND AGE OF GRANDPARENT',
  'Estimate!!Total!!In labor force'),
 ('B10058_003E',
  'EMPLOYMENT STATUS OF GRANDPARENTS LIVING WITH OWN GRANDCHILDREN UNDER 18 YEARS BY RESPONSIBILITY FOR OWN GRANDCHILDREN AND AGE OF GRANDPARENT',
  'Estimate!!Total!!In labor force!!Grandparent responsible for own grandchildren under 18 years'),
 ('B10058_004E',
  'EMPLOYMENT STATUS OF GRANDPARENTS LIVING WITH OWN GRANDCHILDREN UNDER 18 YEARS BY RESPONSIBILITY FOR OWN GRANDCHILDREN AND AGE OF GRANDPARENT',
  'Estimate!!Total!!In labor force!!Grandparent responsible for own grandchildren under 18 years!!30 to 59 years'),
 ('B10058_005E',
  'EMPLOYMENT STATUS OF GRANDPARENTS LIVING WITH OW

In [308]:
# get information on a target variable
censusdata.printtable(censusdata.censustable('acs5', 2018, 'B25008'))
pop_occupied_housing_variables = list(censusdata.censustable('acs5', 2018, 'B25008').keys())

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B25008_001E  | TOTAL POPULATION IN OCCUPIED H | !! Estimate Total                                        | int  
B25008_002E  | TOTAL POPULATION IN OCCUPIED H | !! !! Estimate Total Owner occupied                      | int  
B25008_003E  | TOTAL POPULATION IN OCCUPIED H | !! !! Estimate Total Renter occupied                     | int  
-------------------------------------------------------------------------------------------------------------------


In [309]:
pop_occupied_housing_variables # this can be fed into the acs variable extractor

['B25008_001E', 'B25008_002E', 'B25008_003E']

#### Appendix B - EXTRA FEATURE: Extracting Zip Code Data

In [310]:
# download above housing variable data for all zip codes
pop_occupied_housing_zip = censusdata.download('acs5', 2018, censusdata.censusgeo([('zip code tabulation area', '*')]), pop_occupied_housing_variables) # '*' can be replaced by specific zip code
pop_occupied_housing_zip

,B25008_001E,B25008_002E,B25008_003E
"ZCTA5 43964: Summary level: 860, zip code tabulation area:43964",8608,6300,2308
"ZCTA5 28216: Summary level: 860, zip code tabulation area:28216",50246,27296,22950
"ZCTA5 28277: Summary level: 860, zip code tabulation area:28277",71605,52982,18623
"ZCTA5 28278: Summary level: 860, zip code tabulation area:28278",27286,19498,7788
"ZCTA5 28303: Summary level: 860, zip code tabulation area:28303",29202,13987,15215
...,...,...,...
"ZCTA5 98279: Summary level: 860, zip code tabulation area:98279",628,496,132
"ZCTA5 98280: Summary level: 860, zip code tabulation area:98280",417,394,23
"ZCTA5 98311: Summary level: 860, zip code tabulation area:98311",27657,20346,7311
"ZCTA5 98326: Summary level: 860, zip code tabulation area:98326",493,346,147
